# 서울특별시 자치구 별 지하철 승하차인원 파악

## 목차 

### 1. module import
### 2. 지하철 좌표값을 이용해 해당 지하철이 속한 자치구 파악
### 3. 기간에 따른 지하철별 승하차 인원 확인
<br/>

### 1. module import
- import **`pandas`** & import **`numpy`**
    - csv file 및 dataframe을 가공하기 위함
- import **`requests`** & import **`json`**
    - requests 모듈과 API를 활용하여 웹사이트에 접근하여,
    - 요청과 응답을 통해 json형태로 데이터를 받아온 후,
    - Python을 통해 필요한 데이터를 수집하고 가공하기위함

In [1]:
import pandas as pd
import numpy as np
import requests
import json

### 2. 지하철 좌표값을 이용해 지하철역이 속한 자치구 파악

#### 2.1 pandas를 활용해 csv파일 불러오기 및 dataframe 구조 확인

In [2]:
# 지하철역별 좌표값을 이용해 지하철이 속한 자치구 파악
coor = pd.read_csv("./data/subway/station_coordinate.csv")

In [3]:
# 데이터 확인
coor.head(5)

,line,name,code,lat,lng
0,01호선,녹양,1908.0,37.759380,127.042292
1,01호선,남영,1002.0,37.541021,126.971300
2,01호선,용산,1003.0,37.529849,126.964561
3,01호선,노량진,1004.0,37.514219,126.942454
4,01호선,대방,1005.0,37.513342,126.926382


In [4]:
# 데이터 구조 확인
coor.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 730 entries, 0 to 729
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   line    730 non-null    object 
 1   name    730 non-null    object 
 2   code    727 non-null    float64
 3   lat     700 non-null    float64
 4   lng     700 non-null    float64
dtypes: float64(3), object(2)
memory usage: 28.6+ KB


In [6]:
# info 확인 결과 null이 존재함을 파악
# null 값 확인
coor[coor["lat"].isnull()].head()

,line,name,code,lat,lng
244,05호선,미사,2563.0,NaN,NaN
245,05호선,하남풍산,2564.0,NaN,NaN
401,09호선,석촌고분,4132.0,NaN,NaN
403,09호선,송파나루,4134.0,NaN,NaN
404,09호선,한성백제,4135.0,NaN,NaN


In [7]:
# copy 만들어서 data 가공
my_coor = coor.copy()

In [8]:
# 좌표값 확인
my_coor["lat"].iloc[0]

37.75938

#### 2.2 네이버 클라우드 플랫폼의 Reverse-Geocoding application을 이용하여 좌표별 자치구 파악
- <a style="text-decoration: none; color:black;">[https://console.ncloud.com/naver-service/application]<a/>

In [9]:
# 발급받은 두개의 Application Key 저장
client_id = "8la6jz3flf"
client_secret = "jnuz1aeEGOEDBExyNsfmrfCZco9LDhuSLj36N9mb"
# 지하철역별 자치구를 담을 공간
gu = []

# 좌표값 변수에 담아주기 / lng: 경도(longitude), lat: 위도(latitude)
for i in range(len(my_coor)):
    # 좌표가 없는 곳 걸러내기
    if str(my_coor["lng"].iloc[i]) == "nan":
        gu.append("")
    else:
        coords = str(my_coor["lng"].iloc[i])+","+str(my_coor["lat"].iloc[i])
        # 요청을 보낼 url
        url=f"https://naveropenapi.apigw.ntruss.com/map-reversegeocode/v2/gc?coords={coords}&output=json"
        # headers에 보낼 API-KEY
        headers = {
            "X-NCP-APIGW-API-KEY-ID": client_id,
            "X-NCP-APIGW-API-KEY": client_secret,
        }
        # url과 headers를 통해 서버에 request하고 response를 res에 저장
        res = requests.get(url, headers=headers)
        # json type의 데이터를 dict type으로 바꾸기
        my_dict = res.json()
        # Indexing을 통해 전체 주소 중 자치구만 gu에 추가
        gu.append(my_dict["results"][0]["region"]["area2"]["name"])

In [10]:
# "구" 라는 컬럼을 만들어 각 지하철역이 속하는 자치구 추가
my_coor["구"] = gu

In [11]:
# 반복적인 API요청을 피하기 위해서 train_gu.csv 파일로 저장
my_coor.to_csv("subway_gu.csv")

In [12]:
# 저장한 파일 불러와서 확인
my_coor = pd.read_csv("./data/subway/subway_gu.csv", index_col=0)
my_coor.head()

,line,name,code,lat,lng,구
0,01호선,녹양,1908.0,37.759380,127.042292,의정부시
1,01호선,남영,1002.0,37.541021,126.971300,용산구
2,01호선,용산,1003.0,37.529849,126.964561,용산구
3,01호선,노량진,1004.0,37.514219,126.942454,동작구
4,01호선,대방,1005.0,37.513342,126.926382,영등포구


In [13]:
# 결측치 확인
my_coor.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 730 entries, 0 to 729
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   line    730 non-null    object 
 1   name    730 non-null    object 
 2   code    727 non-null    float64
 3   lat     700 non-null    float64
 4   lng     700 non-null    float64
 5   구       700 non-null    object 
dtypes: float64(3), object(3)
memory usage: 39.9+ KB


### 3. 기간에 따른 지하철역별 승하차 인원 확인

#### 3.1 반복문을 통해 데이터를 변수에 저장
- 설정기간: 19년 1월 ~ 22년 9월

In [14]:
# ./data/train 경로안에 CARD_SUBWAY_MONTH_201901.csv 파일 형태로 저장되어있음
# exec 함수를 사용하여 동적할당을 통해 df201901 형태의 변수로 저장(19년 1월 예시)

# data 상대경로 
url = "./data/subway/CARD_SUBWAY_MONTH_20"
# encoding type
enc_type = "cp949"
enc_type2 = "utf-8"


# 년도별
for i in range(19, 23):
    # 월별
    if i == 22: # 22년도는 9월까지 밖에 없음
        for j in range(1, 10):
            url_temp = ""
            url_temp = f"{url}{i}0{j}.csv"
            try:
                exec(f"df20{i}0{j} = pd.read_csv(url_temp, encoding=enc_type)")
                print(url_temp) # 실행여부 확인
            except:
                exec(f"df20{i}0{j} = pd.read_csv(url_temp, encoding=enc_type2)")
                print(url_temp)
    else:
        for j in range(1, 13):
            url_temp = ""
            if j < 10:
                url_temp = f"{url}{i}0{j}.csv"
                try:
                    exec(f"df20{i}0{j} = pd.read_csv(url_temp, encoding=enc_type)")
                    print(url_temp)
                except:
                    exec(f"df20{i}0{j} = pd.read_csv(url_temp, encoding=enc_type2)")
                    print(url_temp)
            else:
                url_temp = f"{url}{i}{j}.csv"
                try:
                    exec(f"df20{i}{j} = pd.read_csv(url_temp, encoding=enc_type)")
                    print(url_temp)
                except:
                    exec(f"df20{i}{j} = pd.read_csv(url_temp, encoding=enc_type2)")
                    print(url_temp)

./data/subway/CARD_SUBWAY_MONTH_201901.csv
./data/subway/CARD_SUBWAY_MONTH_201902.csv
./data/subway/CARD_SUBWAY_MONTH_201903.csv
./data/subway/CARD_SUBWAY_MONTH_201904.csv
./data/subway/CARD_SUBWAY_MONTH_201905.csv
./data/subway/CARD_SUBWAY_MONTH_201906.csv
./data/subway/CARD_SUBWAY_MONTH_201907.csv
./data/subway/CARD_SUBWAY_MONTH_201908.csv
./data/subway/CARD_SUBWAY_MONTH_201909.csv
./data/subway/CARD_SUBWAY_MONTH_201910.csv
./data/subway/CARD_SUBWAY_MONTH_201911.csv
./data/subway/CARD_SUBWAY_MONTH_201912.csv
./data/subway/CARD_SUBWAY_MONTH_202001.csv
./data/subway/CARD_SUBWAY_MONTH_202002.csv
./data/subway/CARD_SUBWAY_MONTH_202003.csv
./data/subway/CARD_SUBWAY_MONTH_202004.csv
./data/subway/CARD_SUBWAY_MONTH_202005.csv
./data/subway/CARD_SUBWAY_MONTH_202006.csv
./data/subway/CARD_SUBWAY_MONTH_202007.csv
./data/subway/CARD_SUBWAY_MONTH_202008.csv
./data/subway/CARD_SUBWAY_MONTH_202009.csv
./data/subway/CARD_SUBWAY_MONTH_202010.csv
./data/subway/CARD_SUBWAY_MONTH_202011.csv
./data/subw

In [15]:
# 데이터 확인
df202209.head()

,사용일자,노선명,역명,승차총승객수,하차총승객수,등록일자
20220901,2호선,신대방,26990,26203,20220904,NaN
20220901,2호선,신림,61313,58762,20220904,NaN
20220901,2호선,봉천,26070,23873,20220904,NaN
20220901,2호선,서울대입구(관악구청),51225,50545,20220904,NaN
20220901,2호선,낙성대(강감찬),30179,29402,20220904,NaN


#### 3.2 20년 5월부터 21년 10월, 21년 12월부터 22년 9월까지 데이터 수정
- 데이터의 컬럼이 밀려 있음
- 4단계를 거쳐 수정
    1. 등록일자 컬럼 삭제
        - df22_09.drop(["등록일자"], axis=1, inplace = True)
    2. 밀려있는 컬럼명 수정
        - df202209.rename(columns = {"사용일자": "노선명", "노선명": "역명", "역명": "승차총승객수", "승차총승객수": "하차총승객수", "하차총승객수": "등록일자"} , inplace=True)
    3. index를 사용일자 컬럼으로 추가
        - df202209["사용일자"] = df202209.index
    4. reset_index
        - df202209.reset_index(drop=True)

In [16]:
# 데이터 수정 과정
drop_column = ["등록일자"]
new_columns = {"사용일자": "노선명", "노선명": "역명", "역명": "승차총승객수", "승차총승객수": "하차총승객수", "하차총승객수": "등록일자"}
add_column = "사용일자"

# 20년 5월부터 20년 12월
for i in range(5, 13):
    if i < 10:
        exec(f"df20200{i}.drop(drop_column, axis=1, inplace=True)")
        exec(f"df20200{i}.rename(columns = new_columns, inplace=True)")
        exec(f"df20200{i}[add_column] = df20200{i}.index")
        exec(f"df20200{i}.reset_index(drop=True, inplace=True)")
        print(f"df20200{i}") # 실행 여부 확인
    else:
        exec(f"df2020{i}.drop(drop_column, axis=1, inplace=True)")
        exec(f"df2020{i}.rename(columns = new_columns, inplace=True)")
        exec(f"df2020{i}[add_column] = df2020{i}.index")
        exec(f"df2020{i}.reset_index(drop=True, inplace=True)")
        print(f"df2020{i}")

# 21년 1월부터 10월 , 21년 12월
for i in range(1, 13):
    if i < 10:
        exec(f"df20210{i}.drop(drop_column, axis=1, inplace=True)")
        exec(f"df20210{i}.rename(columns = new_columns, inplace=True)")
        exec(f"df20210{i}[add_column] = df20210{i}.index")
        exec(f"df20210{i}.reset_index(drop=True, inplace=True)")
        print(f"df20210{i}")

    elif i == 10 or i == 12:
        exec(f"df2021{i}.drop(drop_column, axis=1, inplace=True)")
        exec(f"df2021{i}.rename(columns = new_columns, inplace=True)")
        exec(f"df2021{i}[add_column] = df2021{i}.index")
        exec(f"df2021{i}.reset_index(drop=True, inplace=True)")
        print(f"df2021{i}")

# 22년 1월부터 22년 9월
for i in range(1, 10):
    exec(f"df20220{i}.drop(drop_column, axis=1, inplace=True)")
    exec(f"df20220{i}.rename(columns = new_columns, inplace=True)")
    exec(f"df20220{i}[add_column] = df20220{i}.index")
    exec(f"df20220{i}.reset_index(drop=True, inplace=True)")
    print(f"df20220{i}")

df202005
df202006
df202007
df202008
df202009
df202010
df202011
df202012
df202101
df202102
df202103
df202104
df202105
df202106
df202107
df202108
df202109
df202110
df202112
df202201
df202202
df202203
df202204
df202205
df202206
df202207
df202208
df202209


In [17]:
# 데이터 확인
df202209.head()

,노선명,역명,승차총승객수,하차총승객수,등록일자,사용일자
0,2호선,신대방,26990,26203,20220904,20220901
1,2호선,신림,61313,58762,20220904,20220901
2,2호선,봉천,26070,23873,20220904,20220901
3,2호선,서울대입구(관악구청),51225,50545,20220904,20220901
4,2호선,낙성대(강감찬),30179,29402,20220904,20220901


#### 3.3 지하철역을 기준으로 작업
1. 역명을 기준으로 groupby 후 승차총승객수, 하차총승객수 합 구하기
2. 불필요 컬럼 제거
3. 역명 통일 시키기
    - 역명 뒤에 ()내용 제거
    - 4.19민주묘지를 419민주묘지로 바꾸기(좌표데이터와 매칭시키기 위해서)
4. 역명을 기준으로 어떤 자치구에 속하는지 추가
    - 좌표와 자치구 데이터가 있는 my_coor 이용
5. 기타 작업

In [18]:
# 역명 기준 groupby, 불필요 컬럼 제거
station = "역명"
drop_columns = ["등록일자", "사용일자"]
new_columns = ["역명", "승차총승객수", "하차총승객수"]
# 년도별
for i in range(19, 23):
    # 월별
    if i == 22: # 22년도는 9월까지
        for j in range(1, 10):
            # 역명을 기준으로 groupby
            exec(f"df20{i}0{j} = df20{i}0{j}.groupby(station, as_index=False).sum()")

            # 불필요 컬럼 제거
            exec(f"df20{i}0{j}.drop(drop_columns, axis=1, inplace=True)")
            
            # 컬럼순서 변경
            exec(f"df20{i}0{j} = df20{i}0{j}[new_columns]")

            print(f"df20{i}0{j}")
    else:
        for j in range(1, 13):
            if j < 10:
                exec(f"df20{i}0{j} = df20{i}0{j}.groupby(station, as_index=False).sum()")
                exec(f"df20{i}0{j}.drop(drop_columns, axis=1, inplace=True)")
                exec(f"df20{i}0{j} = df20{i}0{j}[new_columns]")
                print(f"df20{i}0{j}")
            else:
                exec(f"df20{i}{j} = df20{i}{j}.groupby(station, as_index=False).sum()")
                exec(f"df20{i}{j}.drop(drop_columns, axis=1, inplace=True)")
                exec(f"df20{i}{j} = df20{i}{j}[new_columns]")
                print(f"df20{i}{j}")

df201901
df201902
df201903
df201904
df201905
df201906
df201907
df201908
df201909
df201910
df201911
df201912
df202001
df202002
df202003
df202004
df202005
df202006
df202007
df202008
df202009
df202010
df202011
df202012
df202101
df202102
df202103
df202104
df202105
df202106
df202107
df202108
df202109
df202110
df202111
df202112
df202201
df202202
df202203
df202204
df202205
df202206
df202207
df202208
df202209


In [19]:
# 데이터 확인
df202209.head()

,역명,승차총승객수,하차총승객수
0,4.19민주묘지,112492,107136
1,가능,195572,186563
2,가락시장,448763,469819
3,가산디지털단지,1521622,1593369
4,가양,585342,565108


In [20]:
# 역 중에 회현(남대문시장) 과 같이 ()가 있으면 ()전까지 이름으로 저장
# 4.19민주묘지  -> 419민주묘지로 변경

# 년도별 
for i in range(19, 23):
    # 월별
    if i == 22: # 22년도는 9월까지
        for j in range(1, 10):
            exec(f"data = df20{i}0{j}.copy()")
            for m in range(len(data)):
                for n in range(len(data["역명"].iloc[m])):
                    if data["역명"].iloc[m][n] == "(":
                        data["역명"].iloc[m] = str(data["역명"].iloc[m])[0:n]
                        break
                if data["역명"].iloc[m] == "4.19민주묘지":
                    data["역명"].iloc[m] = "419민주묘지"
            exec(f"df20{i}0{j} = data.copy()")
            
    else:
        for j in range(1, 13):
            if j < 10:
                exec(f"data = df20{i}0{j}.copy()")
                for m in range(len(data)):
                    for n in range(len(data["역명"].iloc[m])):
                        if data["역명"].iloc[m][n] == "(":
                            data["역명"].iloc[m] = str(data["역명"].iloc[m])[0:n]
                            break
                    if data["역명"].iloc[m] == "4.19민주묘지":
                        data["역명"].iloc[m] = "419민주묘지"
                exec(f"df20{i}0{j} = data.copy()")
            else:
                exec(f"data = df20{i}{j}.copy()")
                for m in range(len(data)):
                    for n in range(len(data["역명"].iloc[m])):
                        if data["역명"].iloc[m][n] == "(":
                            data["역명"].iloc[m] = str(data["역명"].iloc[m])[0:n]
                            break
                    if data["역명"].iloc[m] == "4.19민주묘지":
                        data["역명"].iloc[m] = "419민주묘지"
                exec(f"df20{i}{j} = data.copy()")

C:\Users\user\AppData\Local\Temp\ipykernel_34336\2536890473.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["역명"].iloc[m] = "419민주묘지"
C:\Users\user\AppData\Local\Temp\ipykernel_34336\2536890473.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["역명"].iloc[m] = str(data["역명"].iloc[m])[0:n]
C:\Users\user\AppData\Local\Temp\ipykernel_34336\2536890473.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["역명"].iloc[m] 

In [21]:
# 데이터 확인
df202209.head()

,역명,승차총승객수,하차총승객수
0,419민주묘지,112492,107136
1,가능,195572,186563
2,가락시장,448763,469819
3,가산디지털단지,1521622,1593369
4,가양,585342,565108


In [22]:
# 역명에 따른 구 컬럼 추가
station = "역명"

# 년도별
for i in range(19, 23):
    # 월별
    if i == 22: # 22년도는 9월까지
        for j in range(1, 10):
            # 자치구를 담을 리스트
            gu_list = []  
            # 데이터 카피 -> exec 사용을 줄일 수 있음
            exec(f"data = df20{i}0{j}.copy()")
            for k in range(len(data)):
                
                # 역명 추출
                exec(f"name = data[station].iloc[k]")
                
                try: # 해당 역이 어떤 자치구에 속한다면
                    gu = my_coor[my_coor["name"] == name]["구"].iloc[0]
                    gu_list.append(gu)
                except: # 해당 역이 자치구에 속하지 못한다면
                    gu_list.append("")
            data["구"] = gu_list
            exec(f"df20{i}0{j} = data.copy()")
            print(f"df20{i}0{j}") # 실행여부 확인
    else:
        for j in range(1, 13):
            gu_list = []
            if j < 10:
                exec(f"data = df20{i}0{j}.copy()")
                for k in range(len(data)):
                    exec(f"name = data[station].iloc[k]")
                    try:
                        gu = my_coor[my_coor["name"] == name]["구"].iloc[0]
                        gu_list.append(gu)
                    except:
                        gu_list.append("")
                data["구"] = gu_list
                exec(f"df20{i}0{j} = data.copy()")
                print(f"df20{i}0{j}")
            else:
                exec(f"data = df20{i}{j}.copy()")
                for k in range(len(data)):
                    exec(f"name = data[station].iloc[k]")
                    try:
                        gu = my_coor[my_coor["name"] == name]["구"].iloc[0]
                        gu_list.append(gu)
                    except:
                        gu_list.append("")
                data["구"] = gu_list
                exec(f"df20{i}{j} = data.copy()")
                print(f"df20{i}{j}")

df201901
df201902
df201903
df201904
df201905
df201906
df201907
df201908
df201909
df201910
df201911
df201912
df202001
df202002
df202003
df202004
df202005
df202006
df202007
df202008
df202009
df202010
df202011
df202012
df202101
df202102
df202103
df202104
df202105
df202106
df202107
df202108
df202109
df202110
df202111
df202112
df202201
df202202
df202203
df202204
df202205
df202206
df202207
df202208
df202209


In [23]:
# 데이터 확인
df202209.head()

,역명,승차총승객수,하차총승객수,구
0,419민주묘지,112492,107136,강북구
1,가능,195572,186563,의정부시
2,가락시장,448763,469819,송파구
3,가산디지털단지,1521622,1593369,금천구
4,가양,585342,565108,강서구


In [24]:
# 둔촌오륜, 삼전, 석촌고분, 송파나루, 중앙보훈병원, 한성백제 역은 서울시에 속하지만 좌표값이 없음
# 수작업으로 추가해주기
# 둔촌오륜, 삼전, 석촌고분, 송파나루, 한성백제: 송파구 / 중앙보훈병원: 강동구 
df202209[df202209["구"].isnull()]

,역명,승차총승객수,하차총승객수,구
147,둔촌오륜,30057,28271,NaN
181,미사,457028,461868,NaN
233,삼전,190777,179879,NaN
262,석촌고분,174616,168564,NaN
284,송파나루,142192,153884,NaN
418,인천공항1터미널,239329,226674,NaN
419,인천공항2터미널,111171,97999,NaN
424,임진강,1415,1153,NaN
452,중앙보훈병원,289413,258492,NaN
496,하남풍산,115772,113943,NaN


In [25]:
# 좌표값이 없는 역이 속하는 자치구 추가
nan_list = ["둔촌오륜", "삼전", "석촌고분", "송파나루", "한성백제"]

# 년도
for i in range(19, 23):
    if i == 22:
        for j in range(1, 10):
            exec(f"data = df20{i}0{j}.copy()")
            data.loc[data["역명"].isin(nan_list), "구"] = "송파구"
            data.loc[data["역명"] == "중앙보훈병원", "구"] = "강동구"
            exec(f"df20{i}0{j} = data.copy()")
            print(f"df20{i}0{j}")
    else:
        for j in range(1, 13):
            if j < 10:
                exec(f"data = df20{i}0{j}.copy()")
                data.loc[data["역명"].isin(nan_list), "구"] = "송파구"
                data.loc[data["역명"] == "중앙보훈병원", "구"] = "강동구"
                exec(f"df20{i}0{j} = data.copy()")
                print(f"df20{i}0{j}")
            else:
                exec(f"data = df20{i}{j}.copy()")
                data.loc[data["역명"].isin(nan_list), "구"] = "송파구"
                data.loc[data["역명"] == "중앙보훈병원", "구"] = "강동구"
                exec(f"df20{i}{j} = data.copy()")
                print(f"df20{i}{j}")

df201901
df201902
df201903
df201904
df201905
df201906
df201907
df201908
df201909
df201910
df201911
df201912
df202001
df202002
df202003
df202004
df202005
df202006
df202007
df202008
df202009
df202010
df202011
df202012
df202101
df202102
df202103
df202104
df202105
df202106
df202107
df202108
df202109
df202110
df202111
df202112
df202201
df202202
df202203
df202204
df202205
df202206
df202207
df202208
df202209


In [26]:
# 들어간걸 확인
df202201.loc[df202201["역명"] == "송파나루"]

,역명,승차총승객수,하차총승객수,구
281,송파나루,136404,144580,송파구


#### 3.4 자치구별로 작업
1. 자치구별로 그룹화
2. 서울특별시 자치구에 해당하는 구만 추출
3. 해당기간 추가 및 컬럼순서 변경
4. csv 파일로 저장

In [27]:
gu_lists=['강남구', '관악구', '성북구', '서초구', '서대문구', '영등포구', '종로구', '송파구', '동대문구', '마포구', '은평구', '동작구', '구로구', '강북구', '강서구', '용산구', '노원구', '양천구', '중랑구', '중구', '금천구', '도봉구', '성동구', '광진구', '강동구']


# 년도별
for i in range(19, 23):
    # 월별
    if i == 22: # 22년도는 9월까지
        for j in range(1, 10):
            exec(f"data = df20{i}0{j}.copy()")
            # 그룹화는 임의로 하차총승객수 기준으로 시행함
            data = data.groupby("구", as_index=False).sum().sort_values(by=["하차총승객수"], ascending=False).reset_index(drop=True)
            subway = data[data["구"].isin(gu_lists)].reset_index(drop=True)
            
            # 기간 컬럼 추가
            date = f"20{i}0{j}"
            subway["기간"] = date
            
            # 컬럼 순서 변경
            subway = subway[["기간", "구", "승차총승객수", "하차총승객수"]]
            
            # 데이터를 subway202209.csv 파일 형태로 저장
            csv_name = f"subway20{i}0{j}.csv"
            subway.to_csv(csv_name)
            print(f"df20{i}0{j}")
    else:
        for j in range(1, 13):
            if j < 10:
                exec(f"data = df20{i}0{j}.copy()")
                data = data.groupby("구", as_index=False).sum().sort_values(by=["하차총승객수"], ascending=False).reset_index(drop=True)
                subway = data[data["구"].isin(gu_lists)].reset_index(drop=True)
                date = f"20{i}0{j}"
                subway["기간"] = date
                subway = subway[["기간", "구", "승차총승객수", "하차총승객수"]]
                csv_name = f"subway20{i}0{j}.csv"
                subway.to_csv(csv_name)
                print(f"df20{i}0{j}")
            else:
                exec(f"data = df20{i}{j}.copy()")
                data = data.groupby("구", as_index=False).sum().sort_values(by=["하차총승객수"], ascending=False).reset_index(drop=True)
                subway = data[data["구"].isin(gu_lists)].reset_index(drop=True)
                date = f"20{i}{j}"
                subway["기간"] = date
                subway = subway[["기간", "구", "승차총승객수", "하차총승객수"]]
                csv_name = f"subway20{i}{j}.csv"
                subway.to_csv(csv_name)
                print(f"df20{i}{j}")

df201901
df201902
df201903
df201904
df201905
df201906
df201907
df201908
df201909
df201910
df201911
df201912
df202001
df202002
df202003
df202004
df202005
df202006
df202007
df202008
df202009
df202010
df202011
df202012
df202101
df202102
df202103
df202104
df202105
df202106
df202107
df202108
df202109
df202110
df202111
df202112
df202201
df202202
df202203
df202204
df202205
df202206
df202207
df202208
df202209


In [28]:
# 데이터 불러오기

# 경로
url = "./data/subwaysubway/subway20"
# 년도별
for i in range(19, 23):
    # 월별
    if i == 22: 
        for j in range(1, 10):
            url_temp = f"{url}{i}0{j}.csv"
            exec(f"subway20{i}0{j} = pd.read_csv(url_temp, index_col=0)")
            print(url_temp) # 실행여부 확인
    else:
        for j in range(1, 13):
            if j < 10:
                url_temp = f"{url}{i}0{j}.csv"
                exec(f"subway20{i}0{j} = pd.read_csv(url_temp, index_col=0)")
                print(url_temp)
            else:
                url_temp = f"{url}{i}{j}.csv"
                exec(f"subway20{i}{j} = pd.read_csv(url_temp, index_col=0)")
                print(url_temp)

./data/subwaysubway/subway201901.csv
./data/subwaysubway/subway201902.csv
./data/subwaysubway/subway201903.csv
./data/subwaysubway/subway201904.csv
./data/subwaysubway/subway201905.csv
./data/subwaysubway/subway201906.csv
./data/subwaysubway/subway201907.csv
./data/subwaysubway/subway201908.csv
./data/subwaysubway/subway201909.csv
./data/subwaysubway/subway201910.csv
./data/subwaysubway/subway201911.csv
./data/subwaysubway/subway201912.csv
./data/subwaysubway/subway202001.csv
./data/subwaysubway/subway202002.csv
./data/subwaysubway/subway202003.csv
./data/subwaysubway/subway202004.csv
./data/subwaysubway/subway202005.csv
./data/subwaysubway/subway202006.csv
./data/subwaysubway/subway202007.csv
./data/subwaysubway/subway202008.csv
./data/subwaysubway/subway202009.csv
./data/subwaysubway/subway202010.csv
./data/subwaysubway/subway202011.csv
./data/subwaysubway/subway202012.csv
./data/subwaysubway/subway202101.csv
./data/subwaysubway/subway202102.csv
./data/subwaysubway/subway202103.csv
.

In [29]:
# 데이터 확인
subway202209.head()

,기간,구,승차총승객수,하차총승객수
0,202209,강남구,14276524,14865184
1,202209,중구,9370232,9519206
2,202209,서초구,9511020,9514861
3,202209,송파구,8950460,8995306
4,202209,마포구,8454240,8784264


In [30]:
# data를 열 기준으로 병합
subway_data = [subway201901,subway201902,subway201903,subway201904,subway201905,subway201906,subway201907,subway201908,subway201909,subway201910,subway201911,subway201912,
               subway202001,subway202002,subway202003,subway202004,subway202005,subway202006,subway202007,subway202008,subway202009,subway202010,subway202011,subway202012,
               subway202101,subway202102,subway202103,subway202104,subway202105,subway202106,subway202107,subway202108,subway202109,subway202110,subway202111,subway202112,
               subway202201,subway202202,subway202203,subway202204,subway202205,subway202206,subway202207,subway202208,subway202209]
result = pd.concat(subway_data, axis=0).reset_index(drop=True)

In [31]:
# 1125 rows 확인
result

,기간,구,승차총승객수,하차총승객수
0,201901,강남구,16629949.0,17385052.0
1,201901,서초구,12573859.0,12741891.0
2,201901,중구,11808612.0,12003489.0
3,201901,마포구,9891741.0,10154318.0
4,201901,종로구,9950033.0,10057901.0
...,...,...,...,...
1120,202209,중랑구,3034046.0,2938462.0
1121,202209,강북구,2920785.0,2850424.0
1122,202209,양천구,2822031.0,2769031.0
1123,202209,도봉구,2595590.0,2505740.0


In [32]:
# csv 파일로 저장
result.to_csv("subway_result.csv")

#### 기타작업
- 월별로 일수가 제각각이기 때문에 월 평균데이터도 구하기

In [33]:
# 월 평균 구하기(ex 1월: 31로 나눠주기)
# 20년 2월은 28일까지 있음
# 1,3,5,7,8,10,12월: 31일
# 4,6,9,11월: 30일
# test[["승차총승객수"]] = test[["승차총승객수"]] // 30
ls_31 = [1, 3, 5, 7, 8, 10, 12]
ls_30 = [4, 6, 9, 11]

# 년도별
for i in range(19, 23):
    # 월별
    if i == 22: # 22년도는 9월까지
        for j in range(1, 10):
            exec(f"data = subway20{i}0{j}.copy()")
            # 31일까지 있는 달
            if j in ls_31:
                data[["승차총승객수"]] = data[["승차총승객수"]] // 31
                data[["하차총승객수"]] = data[["하차총승객수"]] // 31
                # 기간명 변경
                data["기간"] = f"20{i}0{j}"
            # 30일까지 있는 달
            elif j in ls_30:
                data[["승차총승객수"]] = data[["승차총승객수"]] // 30
                data[["하차총승객수"]] = data[["하차총승객수"]] // 30
                data["기간"] = f"20{i}0{j}"
            # 2월
            elif j == 2:
                data[["승차총승객수"]] = data[["승차총승객수"]] // 28
                data[["하차총승객수"]] = data[["하차총승객수"]] // 28
                data["기간"] = f"20{i}0{j}"
            subway = data.copy()
            csv_name = f"subway20{i}0{j}_avg.csv"
            subway.to_csv(csv_name)
    elif i == 20:
        for j in range(1, 13):
            if j < 10:
                exec(f"data = subway20{i}0{j}.copy()")
                if j in ls_31:
                    data[["승차총승객수"]] = data[["승차총승객수"]] // 31
                    data[["하차총승객수"]] = data[["하차총승객수"]] // 31
                    data["기간"] = f"20{i}0{j}"
                elif j in ls_30:
                    data[["승차총승객수"]] = data[["승차총승객수"]] // 30
                    data[["하차총승객수"]] = data[["하차총승객수"]] // 30
                    data["기간"] = f"20{i}0{j}"
                elif j == 2:
                    data[["승차총승객수"]] = data[["승차총승객수"]] // 29
                    data[["하차총승객수"]] = data[["하차총승객수"]] // 29
                    data["기간"] = f"20{i}0{j}"
                subway = data.copy()
                csv_name = f"subway20{i}0{j}_avg.csv"
                subway.to_csv(csv_name)
            else:
                exec(f"data = subway20{i}{j}.copy()")
                if j in ls_31:
                    data[["승차총승객수"]] = data[["승차총승객수"]] // 31
                    data[["하차총승객수"]] = data[["하차총승객수"]] // 31
                    data["기간"] = f"20{i}{j}"
                elif j in ls_30:
                    data[["승차총승객수"]] = data[["승차총승객수"]] // 30
                    data[["하차총승객수"]] = data[["하차총승객수"]] // 30
                    data["기간"] = f"20{i}{j}"
                subway = data.copy()
                csv_name = f"subway20{i}{j}_avg.csv"
                subway.to_csv(csv_name)
    else:
        for j in range(1, 13):
            if j < 10:
                exec(f"data = subway20{i}0{j}.copy()")
                if j in ls_31:
                    data[["승차총승객수"]] = data[["승차총승객수"]] // 31
                    data[["하차총승객수"]] = data[["하차총승객수"]] // 31
                    data["기간"] = f"20{i}0{j}"
                elif j in ls_30:
                    data[["승차총승객수"]] = data[["승차총승객수"]] // 30
                    data[["하차총승객수"]] = data[["하차총승객수"]] // 30
                    data["기간"] = f"20{i}0{j}"
                elif j == 2:
                    data[["승차총승객수"]] = data[["승차총승객수"]] // 28
                    data[["하차총승객수"]] = data[["하차총승객수"]] // 28
                    data["기간"] = f"20{i}0{j}"
                subway = data.copy()
                csv_name = f"subway20{i}0{j}_avg.csv"
                subway.to_csv(csv_name)
            else:
                exec(f"data = subway20{i}{j}.copy()")
                if j in ls_31:
                    data[["승차총승객수"]] = data[["승차총승객수"]] // 31
                    data[["하차총승객수"]] = data[["하차총승객수"]] // 31
                    data["기간"] = f"20{i}{j}"
                elif j in ls_30:
                    data[["승차총승객수"]] = data[["승차총승객수"]] // 30
                    data[["하차총승객수"]] = data[["하차총승객수"]] // 30
                    data["기간"] = f"20{i}{j}"
                subway = data.copy()
                csv_name = f"subway20{i}{j}_avg.csv"
                subway.to_csv(csv_name)

In [34]:
# 평균별로 저장한 데이터 읽어오기 

# 상대경로
url = "./data/subway_avg/subway"

# 년도별
for i in range(19, 23):
    # 월별
    if i == 22: # 22년도는 9월까지 밖에 없음
        for j in range(1, 10):
            url_temp = f"{url}20{i}0{j}_avg.csv"
            exec(f"subway20{i}0{j} = pd.read_csv(url_temp, index_col=0)")
            print(url_temp)
    else:
        for j in range(1, 13):
            if j < 10:
                url_temp = f"{url}20{i}0{j}_avg.csv"
                exec(f"subway20{i}0{j} = pd.read_csv(url_temp, index_col=0)")
                print(url_temp)
            else:
                url_temp = f"{url}20{i}{j}_avg.csv"
                exec(f"subway20{i}{j} = pd.read_csv(url_temp, index_col=0)")
                print(url_temp)

./data/subway_avg/subway201901_avg.csv
./data/subway_avg/subway201902_avg.csv
./data/subway_avg/subway201903_avg.csv
./data/subway_avg/subway201904_avg.csv
./data/subway_avg/subway201905_avg.csv
./data/subway_avg/subway201906_avg.csv
./data/subway_avg/subway201907_avg.csv
./data/subway_avg/subway201908_avg.csv
./data/subway_avg/subway201909_avg.csv
./data/subway_avg/subway201910_avg.csv
./data/subway_avg/subway201911_avg.csv
./data/subway_avg/subway201912_avg.csv
./data/subway_avg/subway202001_avg.csv
./data/subway_avg/subway202002_avg.csv
./data/subway_avg/subway202003_avg.csv
./data/subway_avg/subway202004_avg.csv
./data/subway_avg/subway202005_avg.csv
./data/subway_avg/subway202006_avg.csv
./data/subway_avg/subway202007_avg.csv
./data/subway_avg/subway202008_avg.csv
./data/subway_avg/subway202009_avg.csv
./data/subway_avg/subway202010_avg.csv
./data/subway_avg/subway202011_avg.csv
./data/subway_avg/subway202012_avg.csv
./data/subway_avg/subway202101_avg.csv
./data/subway_avg/subway2

In [35]:
# 데이터 확인
subway202209.head()

,기간,구,승차총승객수,하차총승객수
0,202209,강남구,475884,495506
1,202209,중구,312341,317306
2,202209,서초구,317034,317162
3,202209,송파구,298348,299843
4,202209,마포구,281808,292808


In [36]:
# data를 열 기준으로 병합
subway_data = [subway201901,subway201902,subway201903,subway201904,subway201905,subway201906,subway201907,subway201908,subway201909,subway201910,subway201911,subway201912,
               subway202001,subway202002,subway202003,subway202004,subway202005,subway202006,subway202007,subway202008,subway202009,subway202010,subway202011,subway202012,
               subway202101,subway202102,subway202103,subway202104,subway202105,subway202106,subway202107,subway202108,subway202109,subway202110,subway202111,subway202112,
               subway202201,subway202202,subway202203,subway202204,subway202205,subway202206,subway202207,subway202208,subway202209]
result = pd.concat(subway_data, axis=0).reset_index(drop=True)

In [37]:
result.head()

,기간,구,승차총승객수,하차총승객수
0,201901,강남구,536449.0,560808.0
1,201901,서초구,405608.0,411028.0
2,201901,중구,380922.0,387209.0
3,201901,마포구,319088.0,327558.0
4,201901,종로구,320968.0,324448.0


In [38]:
# csv파일로 저장
result.to_csv("subway_result_avg.csv")